In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
df_test = pd.read_csv('merged_CSVs.csv')
# df_test_1 = df_test.head(5)
# display(df_test_1)

<ipython-input-2-90b701f92e02>:1: DtypeWarning: Columns (3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267

In [ ]:
num_rows = df_test.shape[0]
print(f"The number of rows is: {num_rows}")

The number of rows is: 700776


In [ ]:
columns_to_print = [
    'total_header_bytes', 'max_header_bytes', 'min_header_bytes',
    'mean_header_bytes', 'mode_header_bytes', 'fwd_total_header_bytes',
    'fwd_mode_header_bytes', 'fwd_init_win_bytes', 'bwd_init_win_bytes',
    'packets_rate', 'packet_IAT_total', 'packets_IAT_median',
    'fwd_packets_IAT_mean', 'bwd_packets_IAT_mode', 'label'
]
row_294096 = df_test[columns_to_print].iloc[294096]  # 294097th row is at index 294096 (since index is zero-based)
print(row_294096)
row_294097 = df_test[columns_to_print].iloc[294097]  # 294097th row is at index 294096 (since index is zero-based)
print(row_294097)

total_header_bytes                       364
max_header_bytes                          32
min_header_bytes                          20
mean_header_bytes                    21.4118
mode_header_bytes                       20.0
fwd_total_header_bytes                   212
fwd_mode_header_bytes                   20.0
fwd_init_win_bytes                      8192
bwd_init_win_bytes                     64000
packets_rate                4.72051102657086
packet_IAT_total          3.6013050079345703
packets_IAT_median                    0.0787
fwd_packets_IAT_mean                  0.4001
bwd_packets_IAT_mode                  0.0795
label                                 Benign
Name: 294096, dtype: object
total_header_bytes                       20
max_header_bytes                         20
min_header_bytes                         20
mean_header_bytes                      20.0
mode_header_bytes                      20.0
fwd_total_header_bytes                   20
fwd_mode_header_bytes            

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

input_csv = "merged_CSVs.csv"
output_csv = "llama-2-cpacket-data.csv"
total_rows = 50000  # Desired number of rows for the final dataset
chunk_size = 10000  # Number of rows to write per chunk

# Function to convert a row into the desired text format for LLaMA-2
def row_to_text(row):
    try:
        # User's message based on network flow data
        user_message = (
            f"The network flow has total header bytes of {row['total_header_bytes']} and "
            f"max header bytes of {row['max_header_bytes']} and min header bytes of {row['min_header_bytes']}. "
            f"The mean header bytes are {float(row['mean_header_bytes']):.2f} while the mode is {row['mode_header_bytes']}. "
            f"The forward total header bytes are {row['fwd_total_header_bytes']} and forward mode header bytes are {row['fwd_mode_header_bytes']}. "
            f"The forward initial window bytes are {row['fwd_init_win_bytes']} and backward initial window bytes are {row['bwd_init_win_bytes']}. "
            f"The packet rate is {float(row['packets_rate']):.2f}. The inter-arrival time total is {float(row['packet_IAT_total']):.10f} "
            f"with a median of {float(row['packets_IAT_median']):.10f}. The forward packet IAT mean is {float(row['fwd_packets_IAT_mean']):.10f} "
            f"while the backward packet IAT mode is {float(row['bwd_packets_IAT_mode']):.10f}. "
            f"Is this network data flow associated with any DDoS attacks?"
        )

        # Model's response (LLM's response)
        model_response = f"This data flow can be categorized as {row['label']}."

        # LLaMA-2 format with <s>[INST] tokens
        formatted_text = f"<s>[INST] {user_message} [/INST] {model_response} </s>"
        return formatted_text

    except ValueError as e:
        print(f"Error processing row: {row}, error: {e}")
        return ""

# Load the entire CSV file into a dataframe
df = df_test

# Get the unique labels and calculate the required number of rows per label
unique_labels = df['label'].unique()
rows_per_label = total_rows // len(unique_labels)  # Equal rows for each label

# Sample rows equally for each label
sampled_dfs = []
for label in unique_labels:
    # Filter rows for the current label
    label_df = df[df['label'] == label]
    # If the label has fewer rows than required, sample with replacement
    sampled_label_df = label_df.sample(n=rows_per_label, replace=True, random_state=42)
    sampled_dfs.append(sampled_label_df)

# Combine all sampled rows into a single dataframe
final_df = pd.concat(sampled_dfs)

# Shuffle the final dataframe to mix the rows
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Open the output file in write mode
with open(output_csv, "w") as output_file:
    output_file.write("text\n")  # Write the header

    # Process the final dataframe in chunks and write to the output CSV
    for i in tqdm(range(0, len(final_df), chunk_size)):
        chunk = final_df.iloc[i:i + chunk_size]

        # Convert each row to the LLaMA-2 format and write to file
        for index, row in chunk.iterrows():
            text = row_to_text(row)
            if text:  # Write only if text is non-empty
                output_file.write(f"{text}\n")

print(f"Successfully written {total_rows} rows to {output_csv}")

  0%|          | 0/5 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 8282, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                               

 20%|██        | 1/5 [00:08<00:32,  8.01s/it]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 18230, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                              

 40%|████      | 2/5 [00:13<00:20,  6.73s/it]

...

Streaming output truncated to the last 5000 lines.
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 38210, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                                            src_port
dst_ip                                                

 80%|████████  | 4/5 [00:25<00:06,  6.18s/it]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 48088, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                              

100%|██████████| 5/5 [00:32<00:00,  6.58s/it]

Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                                            src_port
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 49707, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_i

# Fine Tuning

In [14]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl
!pip install tensorboard

In [21]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [17]:
# Load the output csv into df_final

df_final = pd.read_csv('llama-2-cpacket-data.csv')

In [22]:
# Convert the dataframe to a Hugging Face Dataset

dataset = Dataset.from_pandas(final_df)

In [23]:
# Base model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

In [24]:
# Using QLoRA for 4-bit quantization

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [25]:
# Load Llama 2 model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [26]:
# Define PEFT Parameters

peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [34]:
# Set training parameters

training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  #Increase this depending on use case
    per_device_train_batch_size=2,  # Start small to avoid memory issues
    gradient_accumulation_steps=8,  # Accumulate gradients for larger effective batch size
    optim="paged_adamw_32bit",
    save_steps=100,  # Can adjust based on how frequently you want to save checkpoints
    logging_steps=100,  # Can adjust to reduce logging frequency for larger datasets
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [35]:
# Shuffle dataset to ensure that batches are representative of the entire dataset

dataset = dataset.shuffle(seed=42)

In [36]:
# Split dataset into training and validation
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% validation
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [ ]:
# Initialize SFTTrainer (no evaluation)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,  # Training dataset only
    peft_config=peft_params,
    dataset_text_field="text",  # Original dataset field name
    max_seq_length=None,  # Adjust based on your data
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_path = "/content/drive/MyDrive/cPacket/model"

# Save model and tokenizer to Google Drive
trainer.model.save_pretrained(drive_path)
trainer.tokenizer.save_pretrained(drive_path)

In [ ]:
# Save the model and tokenizer for later use

trainer.model.save_pretrained("llama-2-7b-chat-custom")
trainer.tokenizer.save_pretrained("llama-2-7b-chat-custom")

### Model Evaluation

In [ ]:
# Preprocess the val_dataset for Post-Training Evaluation

# Function to split the prompt and expected response
def preprocess_data(example):
    prompt = example['text'].split("[/INST]")[0] + "[/INST]"  # Prompt
    expected_response = example['text'].split("[/INST]")[1].strip()  # Expected response

    return {
        'prompt': prompt,
        'expected_response': expected_response
    }

# Apply this preprocessing to your validation dataset
val_dataset = val_dataset.map(preprocess_data)

In [ ]:
# Generate responses for Valdation set

from transformers import pipeline

# Initialize text generation pipeline with your fine-tuned model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Generate responses for the validation set
generated_labels = []

for example in val_dataset:
    prompt = example['prompt']  # Use the preprocessed prompt
    result = pipe(f"<s>{prompt}")[0]['generated_text']

    # Extract the generated response (remove the prompt part)
    generated_response = result.split("[/INST]")[1].strip()

    # Extract the label (Attack, Benign, Suspicious) from the generated response
    if "Attack" in generated_response:
        generated_labels.append("Attack")
    elif "Benign" in generated_response:
        generated_labels.append("Benign")
    elif "Suspicious" in generated_response:
        generated_labels.append("Suspicious")
    else:
        generated_labels.append("Unknown")  # Handle cases where no label is found


In [ ]:
# Extract labels from true responses

true_labels = []

for example in val_dataset:
    expected_response = example['expected_response']  # True response

    # Extract the label (Attack, Benign, Suspicious) from the expected response
    if "Attack" in expected_response:
        true_labels.append("Attack")
    elif "Benign" in expected_response:
        true_labels.append("Benign")
    elif "Suspicious" in expected_response:
        true_labels.append("Suspicious")
    else:
        true_labels.append("Unknown")  # Handle cases where no label is found


In [ ]:
# Compare the extracted labels and compute accuracy

from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
accuracy = accuracy_score(true_labels, generated_labels)
print(f"Accuracy: {accuracy}")

# Optionally, print a more detailed classification report
print(classification_report(true_labels, generated_labels, labels=["Attack", "Benign", "Suspicious"]))
